In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.9 MB/s eta 0:00:00


# Разбиение по датасетам (больше не запускать)

In [ ]:
import re
import emoji
import pandas as pd

def clean_text(text):
    text = str(text).lower()
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'[!"№;%:?*()\-=\/\\|@#$^&{}\[\]\'.,~`\n\r\t]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_english(text):
    return bool(re.fullmatch(r"[A-Za-z0-9\s]+", text))

def count_by_rating(df, name):
    print(f"\n{name}")
    print(df['rating'].value_counts().sort_index())

def remove_duplicates_between_datasets(df1, df2):
    common_texts = df1['text'][df1['text'].isin(df2['text'])]
    df1 = df1[~df1['text'].isin(common_texts)]
    df2 = df2[~df2['text'].isin(common_texts)]
    return df1, df2

file_path_ru = "/content/drive/MyDrive/Датасеты/RuReviews.csv"
labels = {"positive": 5, "negative": 1, "neautral": 3}
data_ru = []

with open(file_path_ru, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        words = line.split()
        if words and words[-1] in labels:
            rating = labels[words[-1]]
            text = " ".join(words[:-1])
            data_ru.append([rating, text])

ru = pd.DataFrame(data_ru, columns=["rating", "text"])
ru["text"] = ru["text"].astype(str).apply(clean_text)
ru = ru[ru["text"].str.strip() != ""]
ru = ru[~ru["text"].apply(is_english)]
ru = ru[ru["text"].str.split().str.len() > 1]
ru = ru.drop_duplicates(subset='text', keep='first')

wb = pd.read_csv('/content/drive/MyDrive/Датасеты/WB.csv')
wb['text'] = wb['text'].astype(str).apply(clean_text)
wb = wb[~wb["text"].apply(is_english)]
wb = wb[wb["text"].str.split().str.len() > 3]
wb = wb.dropna(subset=['rating', 'text'])
wb = wb.drop_duplicates(subset='text', keep='first')

ru, wb = remove_duplicates_between_datasets(ru, wb)

count_by_rating(ru, "RuReviews после 1 очистки")
count_by_rating(wb, "WB после 1 очистки")
###########################################################################################
test_pos_ru = ru[ru["rating"] == 5].head(7000)
test_neu_ru = ru[ru["rating"] == 3].head(7000)
test_neg_ru = ru[ru["rating"] == 1].head(7000)

test_df1 = pd.concat([test_pos_ru, test_neu_ru, test_neg_ru], ignore_index=True)
test_df1 = test_df1.sample(frac=1, random_state=42).reset_index(drop=True)
test_df1 = test_df1.drop_duplicates(subset='text', keep='first')
test_df1.to_csv("/content/drive/MyDrive/Датасеты/RuReviews_test_21000.csv", index=False)

ru = ru.drop(test_pos_ru.index)
ru = ru.drop(test_neu_ru.index)
ru = ru.drop(test_neg_ru.index)
count_by_rating(ru, "RuReviews после 2 очистки")
###########################################################################################
test_pos_wb = wb[wb["rating"] == 5].head(7000)
test_neu_wb = wb[wb["rating"] == 3].head(7000)
test_neg_wb = wb[wb["rating"] == 1].head(7000)

test_df2 = pd.concat([test_pos_wb, test_neu_wb, test_neg_wb], ignore_index=True)
test_df2 = test_df2.sample(frac=1, random_state=42).reset_index(drop=True)
test_df2 = test_df2.drop_duplicates(subset='text', keep='first')
test_df2.to_csv("/content/drive/MyDrive/Датасеты/WB_test_21000.csv", index=False)
wb = wb.drop(test_pos_wb.index)
wb = wb.drop(test_neu_wb.index)
wb = wb.drop(test_neg_wb.index)

count_by_rating(wb, "WB после 2 очистки")
###########################################################################################
test_pos_ru = ru[ru["rating"] == 5].head(3500)
test_neu_ru = ru[ru["rating"] == 3].head(3500)
test_neg_ru = ru[ru["rating"] == 1].head(3500)
test_pos_wb = wb[wb["rating"] == 5].head(3500)
test_neu_wb = wb[wb["rating"] == 3].head(3500)
test_neg_wb = wb[wb["rating"] == 1].head(3500)

test_df3 = pd.concat([
    test_pos_ru, test_neu_ru, test_neg_ru,
    test_pos_wb, test_neu_wb, test_neg_wb
], ignore_index=True)

test_df3 = test_df3.sample(frac=1, random_state=42).reset_index(drop=True)
test_df3 = test_df3.drop_duplicates(subset='text', keep='first')
test_df3.to_csv("/content/drive/MyDrive/Датасеты/RuWB_for_final_test_21000.csv", index=False)
wb = wb.drop(test_pos_wb.index)
wb = wb.drop(test_neu_wb.index)
wb = wb.drop(test_neg_wb.index)
ru = ru.drop(test_pos_ru.index)
ru = ru.drop(test_neu_ru.index)
ru = ru.drop(test_neg_ru.index)
count_by_rating(ru, "RuReviews после 3 очистки")
count_by_rating(wb, "WB после 3 очистки")
###########################################################################################
val_pos_ru = ru[ru["rating"] == 5].head(3500)
val_neu_ru = ru[ru["rating"] == 3].head(3500)
val_neg_ru = ru[ru["rating"] == 1].head(3500)
val_pos_wb = wb[wb["rating"] == 5].head(3500)
val_neu_wb = wb[wb["rating"] == 3].head(3500)
val_neg_wb = wb[wb["rating"] == 1].head(3500)

val_df = pd.concat([
    val_pos_ru, val_neu_ru, val_neg_ru,
    val_pos_wb, val_neu_wb, val_neg_wb
], ignore_index=True)

val_df = val_df.sample(frac=1, random_state=42).reset_index(drop=True)
val_df = val_df.drop_duplicates(subset='text', keep='first')
val_df.to_csv("/content/drive/MyDrive/Датасеты/RuWB_validation_21000.csv", index=False)
wb = wb.drop(val_pos_wb.index)
wb = wb.drop(val_neu_wb.index)
wb = wb.drop(val_neg_wb.index)
ru = ru.drop(val_pos_ru.index)
ru = ru.drop(val_neu_ru.index)
ru = ru.drop(val_neg_ru.index)
count_by_rating(ru, "RuReviews после 4 очистки")
count_by_rating(wb, "WB после 4 очистки")
###########################################################################################
train_pos_ru = ru[ru["rating"] == 5]
train_neu_ru = ru[ru["rating"] == 3]
train_neg_ru = ru[ru["rating"] == 1]

pos_deficit = 56000 - len(train_pos_ru)
neu_deficit = 56000 - len(train_neu_ru)
neg_deficit = 56000 - len(train_neg_ru)

train_pos_wb = wb[wb["rating"] == 5].head(pos_deficit)
train_neu_wb = wb[wb["rating"] == 3].head(neu_deficit)
train_neg_wb = wb[wb["rating"] == 1].head(neg_deficit)

train_df4 = pd.concat([
    train_pos_ru, train_neu_ru, train_neg_ru,
    train_pos_wb, train_neu_wb, train_neg_wb
], ignore_index=True)

train_df4 = train_df4.sample(frac=1, random_state=42).reset_index(drop=True)
train_df4 = train_df4.drop_duplicates(subset='text', keep='first')
train_df4.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train_balanced.csv", index=False)
wb = wb.drop(train_pos_wb.index)
wb = wb.drop(train_neu_wb.index)
wb = wb.drop(train_neg_wb.index)
ru = ru.drop(train_pos_ru.index)
ru = ru.drop(train_neu_ru.index)
ru = ru.drop(train_neg_ru.index)
count_by_rating(ru, "RuReviews после 5 очистки")
count_by_rating(wb, "WB после 5 очистки")
###########################################################################################
train_df_35 = train_df4.copy()
neutral_add_35 = wb[wb["rating"] == 3].head(19600)
train_df_35 = pd.concat([train_df_35, neutral_add_35], ignore_index=True)
train_df_35 = train_df_35.sample(frac=1, random_state=42).reset_index(drop=True)
train_df_35 = train_df_35.drop_duplicates(subset='text', keep='first')
train_df_35.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train_35.csv", index=False)
wb = wb.drop(neutral_add_35.index)
count_by_rating(wb, "WB после 6 очистки")
###########################################################################################
train_df_55 = train_df_35.copy()
neutral_add_55 = wb[wb["rating"] == 3].head(11200)
train_df_55 = pd.concat([train_df_55, neutral_add_55], ignore_index=True)
train_df_55 = train_df_55.sample(frac=1, random_state=42).reset_index(drop=True)
train_df_55 = train_df_55.drop_duplicates(subset='text', keep='first')
train_df_55.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train_55.csv", index=False)
wb = wb.drop(neutral_add_55.index)
count_by_rating(wb, "WB после 7 очистки")
###########################################################################################
train_df_75 = train_df_55.copy()
neutral_add_75 = wb[wb["rating"] == 3].head(11200)
train_df_75 = pd.concat([train_df_75, neutral_add_75], ignore_index=True)
train_df_75 = train_df_75.sample(frac=1, random_state=42).reset_index(drop=True)
train_df_75 = train_df_75.drop_duplicates(subset='text', keep='first')
train_df_75.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train_75.csv", index=False)
wb = wb.drop(neutral_add_75.index)
count_by_rating(wb, "WB после 8 очистки")
###########################################################################################
train_df_100 = train_df_75.copy()
neutral_add_100 = wb[wb["rating"] == 3].head(14000)
train_df_100 = pd.concat([train_df_100, neutral_add_100], ignore_index=True)
train_df_100 = train_df_100.sample(frac=1, random_state=42).reset_index(drop=True)
train_df_100 = train_df_100.drop_duplicates(subset='text', keep='first')
train_df_100.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train_100.csv", index=False)
wb = wb.drop(neutral_add_100.index)
count_by_rating(wb, "WB после 9 очистки")
###########################################################################################
print('\n Распределение отзывов по классам в получившихся датасетах \n')
count_by_rating(test_df1, "RuReviews_test_21000.csv")
count_by_rating(test_df2, "WB_test_21000.csv")
count_by_rating(test_df3, "RuWB_for_final_test_21000.csv")
count_by_rating(val_df, "RuWB_validation_21000.csv")
count_by_rating(train_df4, "RuWB_train_balanced.csv")
count_by_rating(train_df_35, "RuWB_train_35.csv")
count_by_rating(train_df_55, "RuWB_train_55.csv")
count_by_rating(train_df_75, "RuWB_train_75.csv")
count_by_rating(train_df_100, "RuWB_train_100.csv")


RuReviews после 1 очистки
rating
1    27724
3    27407
5    27797
Name: count, dtype: int64

WB после 1 очистки
rating
1    116972
3    120905
5    106372
Name: count, dtype: int64

RuReviews после 2 очистки
rating
1    20724
3    20407
5    20797
Name: count, dtype: int64

WB после 2 очистки
rating
1    109972
3    113905
5     99372
Name: count, dtype: int64

RuReviews после 3 очистки
rating
1    17224
3    16907
5    17297
Name: count, dtype: int64

WB после 3 очистки
rating
1    106472
3    110405
5     95872
Name: count, dtype: int64

RuReviews после 4 очистки
rating
1    13724
3    13407
5    13797
Name: count, dtype: int64

WB после 4 очистки
rating
1    102972
3    106905
5     92372
Name: count, dtype: int64

RuReviews после 5 очистки
Series([], Name: count, dtype: int64)

WB после 5 очистки
rating
1    60696
3    64312
5    50169
Name: count, dtype: int64

WB после 6 очистки
rating
1    60696
3    44712
5    50169
Name: count, dtype: int64

WB после 7 очистки
rating
1    606

# Проверка на наличие одинаковых отзывов

In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_train_100.csv')
neu_for_train = pd.read_csv('/content/drive/MyDrive/Датасеты/WB_neutral_new.csv')
val = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_validation_21000.csv')
test1 = pd.read_csv('/content/drive/MyDrive/Датасеты/RuReviews_test_21000.csv')
test2 = pd.read_csv('/content/drive/MyDrive/Датасеты/WB_test_21000.csv')
test3 = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_for_final_test_21000.csv')

def check(df, dataset_name):
    duplicates = df[df.duplicated(subset=['text'], keep=False)]
    num_duplicates = duplicates.shape[0]
    if num_duplicates > 0:
        print(f"Найдено {num_duplicates} дубликатов в {dataset_name}.")
        print(duplicates[['text', 'rating']])
    else:
        print(f"Дубликатов не найдено в {dataset_name}.")
    return num_duplicates

num_for_train = check(train, "train")
num_for_neu_for_train = check(neu_for_train, "neu_for_train")
num_for_val = check(val, "val")
num_for_test1 = check(test1, "test1")
num_for_test2 = check(test2, "test2")
num_for_test3 = check(test3, "test3")

combined_df = pd.concat([train, neu_for_train, val, test1, test2, test3], ignore_index=True)
duplicates = combined_df[combined_df.duplicated(subset='text', keep=False)]

if duplicates.empty:
    print("Нет дубликатов в объединенном датасете")
else:
    print(f"Общее количество дубликатов: {len(duplicates)}")
    print(duplicates)


Дубликатов не найдено в train.
Дубликатов не найдено в neu_for_train.
Дубликатов не найдено в val.
Дубликатов не найдено в test1.
Дубликатов не найдено в test2.
Дубликатов не найдено в test3.
Нет дубликатов в объединенном датасете


# Обучение

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset

class RuWBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

train_df = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_train_55.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_validation_21000.csv')
test_ru_df = pd.read_csv('/content/drive/MyDrive/Датасеты/RuReviews_test_21000.csv')
test_wb_df = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_test_21000.csv')
label_mapping = {1: 0, 3: 1, 5: 2}
train_df['label'] = train_df['rating'].map(label_mapping)
val_df['label'] = val_df['rating'].map(label_mapping)
test_ru_df['label'] = test_ru_df['rating'].map(label_mapping)
test_wb_df['label'] = test_wb_df['rating'].map(label_mapping)
train_texts = train_df['text'].tolist()
y_train = train_df['label'].values
val_texts = val_df['text'].tolist()
y_val = val_df['label'].values
test_ru_texts = test_ru_df['text'].tolist()
y_ru_test = test_ru_df['label'].values
test_wb_texts = test_wb_df['text'].tolist()
y_wb_test = test_wb_df['label'].values

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruBert-base", num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_dataset = RuWBDataset(train_texts, y_train, tokenizer)
val_dataset = RuWBDataset(val_texts, y_val, tokenizer)
test_ru_dataset = RuWBDataset(test_ru_texts, y_ru_test, tokenizer)
test_wb_dataset = RuWBDataset(test_wb_texts, y_wb_test, tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    macro_f1 = f1_score(labels, predictions, average='macro')
    return {"macro_f1": macro_f1}

training_args = TrainingArguments(
    output_dir="./RuBERT_last",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.1,
    logging_dir="./logs",
    load_best_model_at_end=True,
    report_to="none",
    metric_for_best_model="macro_f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("/content/drive/MyDrive/RuBERT_55")
tokenizer.save_pretrained("/content/drive/MyDrive/RuBERT_55")

train_predictions = trainer.predict(train_dataset)
y_train_pred = train_predictions.predictions.argmax(axis=1)
print(classification_report(y_train, y_train_pred))
print("F1 (Train):", round(f1_score(y_train, y_train_pred, average='macro')*100, 2))

val_predictions = trainer.predict(val_dataset)
y_val_pred = val_predictions.predictions.argmax(axis=1)
print(classification_report(y_val, y_val_pred))
print("F1 (Validation):", round(f1_score(y_val, y_val_pred, average='macro')*100, 2))

test_ru_predictions = trainer.predict(test_ru_dataset)
y_test_ru_pred = test_ru_predictions.predictions.argmax(axis=1)
print(classification_report(y_ru_test, y_test_ru_pred))
print("F1 (RuReviwes test):", round(f1_score(y_ru_test, y_test_ru_pred, average='macro')*100, 2))

test_wb_predictions = trainer.predict(test_wb_dataset)
y_test_wb_pred = test_wb_predictions.predictions.argmax(axis=1)
print(classification_report(y_wb_test, y_test_wb_pred))
print("F1 (WB test):", round(f1_score(y_wb_test, y_test_wb_pred, average='macro')*100, 2))

# Тестирование модели RuBERT_balanced

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset

class RuWBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    macro_f1 = f1_score(labels, predictions, average='macro')
    return {"macro_f1": macro_f1}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine-tuned-RuBERT_balanced")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/fine-tuned-RuBERT_balanced", num_labels=3)
model.to(device)
label_mapping = {1: 0, 3: 1, 5: 2}
ru_reviews = pd.read_csv('/content/drive/MyDrive/Датасеты/RuReviews_test_21000.csv', sep=",")
ru_reviews['label'] = ru_reviews['rating'].map(label_mapping)
ru_reviews_texts = ru_reviews['text'].tolist()
ru_reviews_labels = ru_reviews['label'].values
ru_reviews_dataset = RuWBDataset(ru_reviews_texts, ru_reviews_labels, tokenizer)

eval_training_args = TrainingArguments(
    output_dir="./temp_eval",
    per_device_eval_batch_size=32,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=eval_training_args,
    compute_metrics=compute_metrics,
)

ru_reviews_predictions = trainer.predict(ru_reviews_dataset)
ru_reviews_y_pred = ru_reviews_predictions.predictions.argmax(axis=1)
ru_reviews['predicted_label'] = ru_reviews_y_pred

errors_df = ru_reviews[ru_reviews['label'] != ru_reviews['predicted_label']][['rating', 'text']]
errors_df.to_csv('/content/drive/MyDrive/Датасеты/errors_in_RuReviews_test.csv', index=False)
errors_counts = errors_df['rating'].value_counts().sort_index()
print("Количество отзывов, на которых модель ошиблась:")
print(errors_counts)

correct_df = ru_reviews[ru_reviews['label'] == ru_reviews['predicted_label']][['rating', 'text']]
correct_df.to_csv('/content/drive/MyDrive/Датасеты/correct_in_RuReviews_test.csv', index=False)
correct_counts = correct_df['rating'].value_counts().sort_index()
print("Количество отзывов, которые модель определила верно:")
print(correct_counts)

print("Тест RuReviews:")
print(classification_report(ru_reviews_labels, ru_reviews_y_pred))
print("Macro F1:", round(f1_score(ru_reviews_labels, ru_reviews_y_pred, average='macro')*100, 2))


Количество отзывов, на которых модель ошиблась:
rating
1    1738
3    2235
5     810
Name: count, dtype: int64
Количество отзывов, которые модель определила верно:
rating
1    5262
3    4765
5    6190
Name: count, dtype: int64
Тест RuReviews:
              precision    recall  f1-score   support

           0       0.76      0.75      0.75      7000
           1       0.67      0.68      0.68      7000
           2       0.89      0.88      0.89      7000

    accuracy                           0.77     21000
   macro avg       0.77      0.77      0.77     21000
weighted avg       0.77      0.77      0.77     21000

Macro F1: 77.26


## Для теста correct_in_RuReviews_test.csv + очищенный ru_test_correct.csv. Выкинутые сохранены в ru_test_incorrect.csv

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/Датасеты/correct_in_RuReviews_test.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Датасеты/ru_test_correct.csv')

combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df.to_csv('/content/drive/MyDrive/Датасеты/RuReviews_test_clean.csv', index=False)
correct_counts = combined_df['rating'].value_counts().sort_index()
print(correct_counts)

df3 = pd.read_csv('/content/drive/MyDrive/Датасеты/ru_test_incorrect.csv')
incorrect_counts = df3['rating'].value_counts().sort_index()
print(incorrect_counts)

rating
1    6548
3    5094
5    6573
Name: count, dtype: int64
rating
1     452
3    1905
5     428
Name: count, dtype: int64


In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset

class RuWBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    macro_f1 = f1_score(labels, predictions, average='macro')
    return {"macro_f1": macro_f1}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine-tuned-RuBERT_balanced")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/fine-tuned-RuBERT_balanced", num_labels=3)
model.to(device)
label_mapping = {1: 0, 3: 1, 5: 2}
ru_reviews = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_validation_21000.csv', sep=",")
ru_reviews['label'] = ru_reviews['rating'].map(label_mapping)
ru_reviews_texts = ru_reviews['text'].tolist()
ru_reviews_labels = ru_reviews['label'].values
ru_reviews_dataset = RuWBDataset(ru_reviews_texts, ru_reviews_labels, tokenizer)

eval_training_args = TrainingArguments(
    output_dir="./temp_eval",
    per_device_eval_batch_size=32,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=eval_training_args,
    compute_metrics=compute_metrics,
)

ru_reviews_predictions = trainer.predict(ru_reviews_dataset)
ru_reviews_y_pred = ru_reviews_predictions.predictions.argmax(axis=1)
ru_reviews['predicted_label'] = ru_reviews_y_pred

errors_df = ru_reviews[ru_reviews['label'] != ru_reviews['predicted_label']][['rating', 'text']]
errors_df.to_csv('/content/drive/MyDrive/Датасеты/errors_in_validation.csv', index=False)
errors_counts = errors_df['rating'].value_counts().sort_index()
print("Количество отзывов, на которых модель ошиблась:")
print(errors_counts)

correct_df = ru_reviews[ru_reviews['label'] == ru_reviews['predicted_label']][['rating', 'text']]
correct_df.to_csv('/content/drive/MyDrive/Датасеты/correct_in_validation.csv', index=False)
correct_counts = correct_df['rating'].value_counts().sort_index()
print("Количество отзывов, которые модель определила верно:")
print(correct_counts)

print("Тест валидациии:")
print(classification_report(ru_reviews_labels, ru_reviews_y_pred))
print("Macro F1:", round(f1_score(ru_reviews_labels, ru_reviews_y_pred, average='macro')*100, 2))


Количество отзывов, на которых модель ошиблась:
rating
1    1662
3    2078
5     690
Name: count, dtype: int64
Количество отзывов, которые модель определила верно:
rating
1    5338
3    4922
5    6310
Name: count, dtype: int64
Тест валидациии:
              precision    recall  f1-score   support

           0       0.77      0.76      0.77      7000
           1       0.70      0.70      0.70      7000
           2       0.90      0.90      0.90      7000

    accuracy                           0.79     21000
   macro avg       0.79      0.79      0.79     21000
weighted avg       0.79      0.79      0.79     21000

Macro F1: 78.92


## Для валидации correct_in_validation.csv + очищенный correct_val.csv. Выкинутые сохранены в incorrect_val.csv

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/Датасеты/correct_in_validation.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Датасеты/correct_val.csv')

combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df.to_csv('/content/drive/MyDrive/Датасеты/Validation_clean.csv', index=False)
correct_counts = combined_df['rating'].value_counts().sort_index()
print(correct_counts)

df3 = pd.read_csv('/content/drive/MyDrive/Датасеты/incorrect_val.csv')
incorrect_counts = df3['rating'].value_counts().sort_index()
print(incorrect_counts)

rating
1    6589
3    5198
5    6592
Name: count, dtype: int64
rating
1     411
3    1802
5     408
Name: count, dtype: int64


# Уникальные нейтральные отзывы для создания трейна +150 и +200

In [ ]:
import re
import emoji
import pandas as pd

def clean_text(text):
    text = str(text).lower()
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'[!"№;%:?*()\-=\/\\|@#$^&{}\[\]\'.,~`\n\r\t]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_english(text):
    return bool(re.fullmatch(r"[A-Za-z0-9\s]+", text))

def count_by_rating(df, name):
    print(f"\n{name}")
    print(df['rating'].value_counts().sort_index())

def remove_duplicates_between_datasets(df1, df2):
    common_texts = df1['text'][df1['text'].isin(df2['text'])]
    df1 = df1[~df1['text'].isin(common_texts)]
    df2 = df2[~df2['text'].isin(common_texts)]
    return df1, df2

wb_neutral_new_path = '/content/drive/MyDrive/Датасеты/WB_neutral_new.csv'

wb = pd.read_csv('/content/drive/MyDrive/Датасеты/WB.csv')
wb['text'] = wb['text'].astype(str).apply(clean_text)
wb = wb[~wb["text"].apply(is_english)]
wb = wb[wb["text"].str.split().str.len() > 3]
wb = wb.dropna(subset=['rating', 'text'])
wb = wb.drop_duplicates(subset='text', keep='first')
count_by_rating(wb, "WB")

wb_300 = pd.read_csv('/content/drive/MyDrive/Датасеты/WB_300.csv')
wb_300['text'] = wb_300['text'].astype(str).apply(clean_text)
wb_300 = wb_300[~wb_300["text"].apply(is_english)]
wb_300 = wb_300[wb_300["text"].str.split().str.len() > 3]
wb_300 = wb_300.dropna(subset=['rating', 'text'])
wb_300 = wb_300.drop_duplicates(subset='text', keep='first')
count_by_rating(wb_300, "WB_300")

wb_300, wb = remove_duplicates_between_datasets(wb_300, wb)
wb_300.to_csv(wb_neutral_new_path, index=False)
new_dataset = pd.read_csv(wb_neutral_new_path)
count_by_rating(new_dataset, "WB_neutral_new")

combined_df = pd.concat([wb, new_dataset])
duplicates_in_combined = combined_df[combined_df.duplicated(subset='text', keep=False)]

if duplicates_in_combined.empty:
    print("В старом WB и новом WB_neutral_new нет повторяющихся отзывов")
else:
    print("В старом WB и новом WB_neutral_new есть повторяющиеся отзывы:")
    print(duplicates_in_combined)


WB
rating
1    117059
3    120973
5    106456
Name: count, dtype: int64

WB_300
rating
3    424014
Name: count, dtype: int64

WB_neutral_new
rating
3    350648
Name: count, dtype: int64
В старом WB и новом WB_neutral_new нет повторяющихся отзывов


In [ ]:
import pandas as pd

train_100 = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_train_100.csv')
neutral_new = pd.read_csv('/content/drive/MyDrive/Датасеты/WB_neutral_new.csv')

train_150 = train_100.copy()
neutral_add_150 = neutral_new[neutral_new["rating"] == 3].head(28000)
train_150 = pd.concat([train_150, neutral_add_150], ignore_index=True)
train_150 = train_150.sample(frac=1, random_state=42).reset_index(drop=True)
train_150 = train_150.drop_duplicates(subset='text', keep='first')
train_150.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train_150.csv", index=False)
neutral_new = neutral_new.drop(neutral_add_150.index)
###########################################################################################
train_200 = train_150.copy()
neutral_add_200 = neutral_new[neutral_new["rating"] == 3].head(28000)
train_200 = pd.concat([train_200, neutral_add_200], ignore_index=True)
train_200 = train_200.sample(frac=1, random_state=42).reset_index(drop=True)
train_200 = train_200.drop_duplicates(subset='text', keep='first')
train_200.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train_200.csv", index=False)
neutral_new = neutral_new.drop(neutral_add_200.index)
###########################################################################################
correct_counts_150 = train_150['rating'].value_counts().sort_index()
print(correct_counts_150)
correct_counts_200 = train_200['rating'].value_counts().sort_index()
print(correct_counts_200)

rating
1     56000
3    140000
5     56000
Name: count, dtype: int64
rating
1     56000
3    168000
5     56000
Name: count, dtype: int64


# Проверка на наличие дубликатов в новых трейнах

In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_train_200.csv')
val = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_validation_21000.csv')
test1 = pd.read_csv('/content/drive/MyDrive/Датасеты/RuReviews_test_21000.csv')
test2 = pd.read_csv('/content/drive/MyDrive/Датасеты/WB_test_21000.csv')
test3 = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_for_final_test_21000.csv')

def check(df, dataset_name):
    duplicates = df[df.duplicated(subset=['text'], keep=False)]
    num_duplicates = duplicates.shape[0]
    if num_duplicates > 0:
        print(f"Найдено {num_duplicates} дубликатов в {dataset_name}.")
        print(duplicates[['text', 'rating']])
    else:
        print(f"Дубликатов не найдено в {dataset_name}.")
    return num_duplicates

num_for_train = check(train, "train")
num_for_val = check(val, "val")
num_for_test1 = check(test1, "test1")
num_for_test2 = check(test2, "test2")
num_for_test3 = check(test3, "test3")

combined_df = pd.concat([train, val, test1, test2, test3], ignore_index=True)
duplicates = combined_df[combined_df.duplicated(subset='text', keep=False)]

if duplicates.empty:
    print("Нет дубликатов в объединенном датасете")
else:
    print(f"Общее количество дубликатов: {len(duplicates)}")
    print(duplicates)


Дубликатов не найдено в train.
Дубликатов не найдено в val.
Дубликатов не найдено в test1.
Дубликатов не найдено в test2.
Дубликатов не найдено в test3.
Нет дубликатов в объединенном датасете
